In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import itertools
import random
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
import itertools
import sklearn.utils
from sklearn.preprocessing import StandardScaler
from statannot import add_stat_annotation
from scipy.stats import ranksums
from statsmodels.stats.multitest import multipletests

In [ ]:
df_crispr=pd.read_csv("/Volumes/SSD_Yue/DepMap_Downloads/CRISPR_gene_effect_22Q2.csv")
df_crispr.index=df_crispr.iloc[:, 0]
df_crispr.drop('DepMap_ID', axis=1, inplace=True)
df_crispr.columns=[i.split(' (')[0] for i in df_crispr.columns]

In [ ]:
h=pd.read_csv("/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/7/h.tsv", sep='\t', index_col=0)
w=pd.read_csv("/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_524_genes_selected_using_pan_cancer_cell_lines_including_MYC_on_30_updated_TNBC_cell_lines_normalized_for_samples_12222022/7/w.tsv", sep='\t', index_col=0)

In [ ]:
cell_lines_order = ['ACH-000691', 'ACH-000196', 'ACH-000111', 'ACH-000276', 'ACH-000721', 'ACH-000148', 'ACH-001389', 'ACH-000621', 'ACH-000288', 'ACH-000573', 'ACH-000699', 'ACH-001391',
                   'ACH-001392', 'ACH-001819', 'ACH-000910', 'ACH-000374', 'ACH-001388', 'ACH-001390', 'ACH-001394',
                   'ACH-000643', 'ACH-000624', 'ACH-000857', 'ACH-000223', 'ACH-000668', 'ACH-000258',
                   'ACH-000856', 'ACH-000212', 'ACH-000768', 'ACH-000849', 'ACH-000536']

In [ ]:
df1=h.apply(lambda x: x.argmax(), axis=0).to_frame()
df1.columns=['state']
df1['state']=df1['state'].astype(str)
df1['states']= df1['state'].map({'0': 'F0', '1': 'F1', '2': 'F2', '3': 'F3', '4': 'F4', '5': 'F5', '6': 'F6', '7': 'F7', '8': 'F8'})
dict_cell_lines={}
for i in w.columns.tolist():
    dict_cell_lines[i]=df1[df1.states==i].index.tolist()

In [ ]:
df_crispr_tnbc = df_crispr.loc[df_crispr.index.isin(cell_lines_order),]

In [ ]:
list_group=[]
for i in df_crispr_tnbc.index.tolist():
    if i in dict_cell_lines['F0']:
        list_group.append('F0')
    else:
        list_group.append('other groups')
df_crispr_tnbc['group']=list_group

In [ ]:
F0_results = []

# Loop through each test (column) and perform the Wilcoxon rank-sum test
for i in ["RBPJ", "MIB1"]:
    stat, p_val = ranksums(df_crispr_tnbc.loc[df_crispr_tnbc.group == "F0", i], df_crispr_tnbc.loc[df_crispr_tnbc.group == "other groups", i], alternative="less")
    # Append results as a dictionary
    F0_results.append({'State': "F0",'Gene': i, 'P-Value': p_val, 'Statistic': stat})
F0_results_df = pd.DataFrame(F0_results)
reject, corrected_p_values, _, _ = multipletests(F0_results_df['P-Value'].tolist(), alpha=0.05, method='fdr_bh')
F0_results_df['p_val_adj']=corrected_p_values.tolist()

In [ ]:
F0_results_df

In [ ]:
df_tmpt_all = pd.DataFrame(columns=['Value', 'Gene', 'Group'])
for i in ["RBPJ", "MIB1"]:
    plt.figure()
    plt.figure(figsize=(8, 6))

    df_tmpt = df_crispr_tnbc[[i]]
    df_tmpt['Gene']=i
    #df_tmpt = df_tmpt.replace([np.inf, -np.inf], np.nan).dropna()
    df_tmpt['flag'] = df_tmpt.index.isin(["ACH-000691", "ACH-000196", "ACH-000276", "ACH-000111"]).astype(int)
    df_tmpt.columns= ["Value", "Gene", "Group"]
    df_tmpt_all = df_tmpt_all.append(df_tmpt, ignore_index=True)
    #x= df_tmpt.iloc[:,0]
    #y= df_tmpt.iloc[:,1]
sns.boxplot(data=df_tmpt_all, x='Gene', y='Value', hue='Group')
    #plt.scatter(x, y, color='blue', marker='o')
    #slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Generate predicted y-values for the regression line
    #regression_line = slope * x + intercept

# Plot regression line
    #plt.plot(x, regression_line, color='red', label="Regression Line")
plt.title('F0 essential genes')
plt.xlabel('Gene')
plt.ylabel('CRISPR score')
plt.grid(False)
#plt.savefig("/Volumes/SSD_Yue/TNBC_paper_Fig2_10252024/F0_CRISPR_essential_genes_MIB1_11032024.svg", format="svg")

In [ ]:
list_group=[]
for i in df_crispr_tnbc.index.tolist():
    if i in dict_cell_lines['F1']:
        list_group.append('F1')
    else:
        list_group.append('other groups')
df_crispr_tnbc['group']=list_group

In [ ]:
F1_results = []

# Loop through each test (column) and perform the Wilcoxon rank-sum test
for i in ["CDH2", "ITGAV"]:
    stat, p_val = ranksums(df_crispr_tnbc.loc[df_crispr_tnbc.group == "F1", i], df_crispr_tnbc.loc[df_crispr_tnbc.group == "other groups", i], alternative="less")
    # Append results as a dictionary
    F1_results.append({'State': "F1",'Gene': i, 'P-Value': p_val, 'Statistic': stat})
F1_results_df = pd.DataFrame(F1_results)
reject, corrected_p_values, _, _ = multipletests(F1_results_df['P-Value'].tolist(), alpha=0.05, method='fdr_bh')
F1_results_df['p_val_adj']=corrected_p_values.tolist()

In [ ]:
F1_results_df

In [ ]:
df_tmpt_all = pd.DataFrame(columns=['Value', 'Gene', 'Group'])
for i in ["CDH2", "ITGAV"]:
    plt.figure()
    plt.figure(figsize=(8, 6))

    df_tmpt = df_crispr_tnbc[[i]]
    df_tmpt['Gene']=i
    #df_tmpt = df_tmpt.replace([np.inf, -np.inf], np.nan).dropna()
    df_tmpt['flag'] = df_tmpt.index.isin(["ACH-000721", "ACH-000148", "ACH-001389", "ACH-000621", "ACH-000288", "ACH-000573", "ACH-000699", "ACH-001391"]).astype(int)
    df_tmpt.columns= ["Value", "Gene", "Group"]
    df_tmpt_all = df_tmpt_all.append(df_tmpt, ignore_index=True)
    #x= df_tmpt.iloc[:,0]
    #y= df_tmpt.iloc[:,1]
sns.boxplot(data=df_tmpt_all, x='Gene', y='Value', hue='Group')
    #plt.scatter(x, y, color='blue', marker='o')
    #slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Generate predicted y-values for the regression line
    #regression_line = slope * x + intercept

# Plot regression line
    #plt.plot(x, regression_line, color='red', label="Regression Line")
plt.title('F1 essential genes')
plt.xlabel('Gene')
plt.ylabel('CRISPR score')
plt.grid(False)
#plt.savefig("/Volumes/SSD_Yue/TNBC_paper_Fig2_10252024/F1_CRISPR_essential_genes_10252024.svg", format="svg")

In [ ]:
list_group=[]
for i in df_crispr_tnbc.index.tolist():
    if i in dict_cell_lines['F2']:
        list_group.append('F2')
    else:
        list_group.append('other groups')
df_crispr_tnbc['group']=list_group

In [ ]:
F2_results = []

# Loop through each test (column) and perform the Wilcoxon rank-sum test
for i in ["FOXA1", "GATA3", "PIK3CA"]:
    stat, p_val = ranksums(df_crispr_tnbc.loc[df_crispr_tnbc.group == "F2", i], df_crispr_tnbc.loc[df_crispr_tnbc.group == "other groups", i], alternative="less")
    # Append results as a dictionary
    F2_results.append({'State': "F2",'Gene': i, 'P-Value': p_val, 'Statistic': stat})
F2_results_df = pd.DataFrame(F2_results)
reject, corrected_p_values, _, _ = multipletests(F2_results_df['P-Value'].tolist(), alpha=0.05, method='fdr_bh')
F2_results_df['p_val_adj']=corrected_p_values.tolist()

In [ ]:
F2_results_df

In [ ]:
df_tmpt_all = pd.DataFrame(columns=['Value', 'Gene', 'Group'])
for i in ["PIK3CA","FOXA1", "GATA3"]:
    plt.figure()
    plt.figure(figsize=(8, 6))

    df_tmpt = df_crispr_tnbc[[i]]
    df_tmpt['Gene']=i
    #df_tmpt = df_tmpt.replace([np.inf, -np.inf], np.nan).dropna()
    df_tmpt['flag'] = df_tmpt.index.isin(["ACH-001932", "ACH-001819", "ACH-000910"]).astype(int)
    df_tmpt.columns= ["Value", "Gene", "Group"]
    df_tmpt_all = df_tmpt_all.append(df_tmpt, ignore_index=True)
    #x= df_tmpt.iloc[:,0]
    #y= df_tmpt.iloc[:,1]
sns.boxplot(data=df_tmpt_all, x='Gene', y='Value', hue='Group')
    #plt.scatter(x, y, color='blue', marker='o')
    #slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Generate predicted y-values for the regression line
    #regression_line = slope * x + intercept

# Plot regression line
    #plt.plot(x, regression_line, color='red', label="Regression Line")
plt.title('F2 essential genes')
plt.xlabel('Gene')
plt.ylabel('CRISPR score')
plt.grid(False)
#plt.savefig("/Volumes/SSD_Yue/TNBC_paper_Fig2_10252024/F2_CRISPR_essential_genes_10252024.svg", format="svg")

In [ ]:
list_group=[]
for i in df_crispr_tnbc.index.tolist():
    if i in dict_cell_lines['F3']:
        list_group.append('F3')
    else:
        list_group.append('other groups')
df_crispr_tnbc['group']=list_group

In [ ]:
F3_results = []

# Loop through each test (column) and perform the Wilcoxon rank-sum test
for i in ["EGFR", "TSG101", "TRAF2"]:
    stat, p_val = ranksums(df_crispr_tnbc.loc[df_crispr_tnbc.group == "F3", i], df_crispr_tnbc.loc[df_crispr_tnbc.group == "other groups", i], alternative="less")
    # Append results as a dictionary
    F3_results.append({'State': "F3",'Gene': i, 'P-Value': p_val, 'Statistic': stat})
F3_results_df = pd.DataFrame(F3_results)
reject, corrected_p_values, _, _ = multipletests(F3_results_df['P-Value'].tolist(), alpha=0.05, method='fdr_bh')
F3_results_df['p_val_adj']=corrected_p_values.tolist()

In [ ]:
F3_results_df

In [ ]:
df_tmpt_all = pd.DataFrame(columns=['Value', 'Gene', 'Group'])
for i in ["EGFR", "TSG101", "TRAF2"]:
    plt.figure()
    plt.figure(figsize=(8, 6))

    df_tmpt = df_crispr_tnbc[[i]]
    df_tmpt['Gene']=i
    #df_tmpt = df_tmpt.replace([np.inf, -np.inf], np.nan).dropna()
    df_tmpt['flag'] = df_tmpt.index.isin(["ACH-000374", "ACH-001388", "ACH-001390", "ACH-001394", "ACH-000643","ACH-000624", "ACH-000857", "ACH-000223", "ACH-000668"]).astype(int)
    df_tmpt.columns= ["Value", "Gene", "Group"]
    df_tmpt_all = df_tmpt_all.append(df_tmpt, ignore_index=True)
    #x= df_tmpt.iloc[:,0]
    #y= df_tmpt.iloc[:,1]
sns.boxplot(data=df_tmpt_all, x='Gene', y='Value', hue='Group')
    #plt.scatter(x, y, color='blue', marker='o')
    #slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Generate predicted y-values for the regression line
    #regression_line = slope * x + intercept

# Plot regression line
    #plt.plot(x, regression_line, color='red', label="Regression Line")
plt.title('F3 essential genes')
plt.xlabel('Gene')
plt.ylabel('CRISPR score')
plt.grid(False)
#plt.savefig("/Volumes/SSD_Yue/TNBC_paper_Fig2_10252024/F3_CRISPR_essential_genes_10252024.svg", format="svg")

In [ ]:
list_group=[]
for i in df_crispr_tnbc.index.tolist():
    if i in dict_cell_lines['F5']:
        list_group.append('F5')
    else:
        list_group.append('other groups')
df_crispr_tnbc['group']=list_group

In [ ]:
F5_results = []

# Loop through each test (column) and perform the Wilcoxon rank-sum test
for i in ["PPAT", "PAICS", "ATIC"]:
    stat, p_val = ranksums(df_crispr_tnbc.loc[df_crispr_tnbc.group == "F5", i], df_crispr_tnbc.loc[df_crispr_tnbc.group == "other groups", i], alternative="less")
    # Append results as a dictionary
    F5_results.append({'State': "F5",'Gene': i, 'P-Value': p_val, 'Statistic': stat})
F5_results_df = pd.DataFrame(F5_results)
reject, corrected_p_values, _, _ = multipletests(F5_results_df['P-Value'].tolist(), alpha=0.05, method='fdr_bh')
F5_results_df['p_val_adj']=corrected_p_values.tolist()

In [ ]:
F5_results_df

In [ ]:
df_tmpt_all = pd.DataFrame(columns=['Value', 'Gene', 'Group'])
for i in ["PPAT", "PAICS", "ATIC"]:
    plt.figure()
    plt.figure(figsize=(8, 6))

    df_tmpt = df_crispr_tnbc[[i]]
    df_tmpt['Gene']=i
    #df_tmpt = df_tmpt.replace([np.inf, -np.inf], np.nan).dropna()
    df_tmpt['flag'] = df_tmpt.index.isin(["ACH-000856", "ACH-000212", "ACH-000768"]).astype(int)
    df_tmpt.columns= ["Value", "Gene", "Group"]
    df_tmpt_all = df_tmpt_all.append(df_tmpt, ignore_index=True)
    #x= df_tmpt.iloc[:,0]
    #y= df_tmpt.iloc[:,1]
sns.boxplot(data=df_tmpt_all, x='Gene', y='Value', hue='Group')
    #plt.scatter(x, y, color='blue', marker='o')
    #slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Generate predicted y-values for the regression line
    #regression_line = slope * x + intercept

# Plot regression line
    #plt.plot(x, regression_line, color='red', label="Regression Line")
plt.title('F5 essential genes')
plt.xlabel('Gene')
plt.ylabel('CRISPR score')
plt.grid(False)
#plt.savefig("/Volumes/SSD_Yue/TNBC_paper_Fig2_10252024/F5_CRISPR_essential_genes_10252024.svg", format="svg")